In [1]:
from keras.models import Model
from keras.layers import LSTM, Dense, Input, Embedding, Concatenate,InputSpec
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K
from keras.engine.topology import Layer
from scipy import stats
from keras.optimizers import Adam
import numpy as np
from keras.layers import Dense, LSTM, Input, Embedding, GlobalAveragePooling1D, Concatenate, Activation, Lambda, BatchNormalization, Convolution1D, Dropout

Using TensorFlow backend.


In [2]:
import pandas
import itertools
import re
import os
from collections import Counter
from sklearn.model_selection import train_test_split

In [21]:
class Neural_Tensor_layer(Layer):
    def __init__(self,output_dim,input_dim=None, **kwargs):
        self.output_dim=output_dim
        self.input_dim=input_dim
        if self.input_dim:
            kwargs['input_shape']=(self.input_dim,)
        super(Neural_Tensor_layer,self).__init__(**kwargs)

    def build(self,input_shape):
        mean=0.0
        std=1.0
        k=self.output_dim
        d=self.input_dim
        W=stats.truncnorm.rvs(-2 * std, 2 * std, loc=mean, scale=std, size=(k,d,d))
        V=stats.truncnorm.rvs(-2 * std, 2 * std, loc=mean, scale=std, size=(2*d,k))
        self.W=K.variable(W)
        self.V=K.variable(V)
        self.b=K.zeros((self.input_dim,))
        self.trainable_weights=[self.W,self.V,self.b]

    def call(self,inputs,mask=None):
        e1=inputs[0]
        e2=inputs[1]
        batch_size=K.shape(e1)[0]
        k=self.output_dim
        feed_forward=K.dot(K.concatenate([e1,e2]),self.V)
        bilinear_tensor_products = [ K.sum((e2 * K.dot(e1, self.W[0])) + self.b, axis=1) ]
        for i in range(k)[1:]:	
            btp=K.sum((e2*K.dot(e1,self.W[i]))+self.b,axis=1)
            bilinear_tensor_products.append(btp)
        result=K.tanh(K.reshape(K.concatenate(bilinear_tensor_products,axis=0),(batch_size,k))+feed_forward)
        return result

    def compute_output_shape(self, input_shape):
        batch_size=input_shape[0][0]
        return(batch_size,self.output_dim)

In [4]:
class Temporal_Mean_Pooling(Layer): 
    def __init__(self, **kwargs):
        self.input_spec=InputSpec(ndim=3)
        super(Temporal_Mean_Pooling,self).__init__(**kwargs)

    def call(self,x):
        mask=K.mean(K.ones_like(x),axis=-1)
        return K.sum(x,axis=-2)/K.sum(mask,axis=-1,keepdims=True)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[2])

In [19]:
def create_model(input_dim = 300,lstm_op_dim=50,vocab_size=4000,embed_dim=300,max_len=1000,coherence_width=50,k=6, start=3):
#     inputs = Input(shape=(max_len,))
#     embedding = Embedding(vocab_size, 300, input_length=max_len)(inputs)
#     lstm = LSTM(lstm_op_dim, return_sequences = True)(embedding)
    inputs = Input(shape=(max_len,input_dim))
    lstm = LSTM(lstm_op_dim, return_sequences = True)(inputs)    
    bilinear_products=Neural_Tensor_layer(output_dim=k,input_dim=lstm_op_dim)
    pairs = [((start + i * coherence_width) % max_len, (start + i * coherence_width + coherence_width) % max_len) for i in range(int(max_len/coherence_width))]
    similarity_pairs = [ (Lambda(lambda t: t[:, p[0], :])(lstm), Lambda(lambda t: t[:, p[1], :])(lstm)) for p in pairs]
    sigmoid_layer = Dense(1, activation="sigmoid")
    similarities = [sigmoid_layer(bilinear_products([w[0], w[1]])) for w in similarity_pairs]  
    tmp = Temporal_Mean_Pooling()(lstm)
    simi = Concatenate()([i for i in similarities])
    tmp_simi = Concatenate()([tmp,simi])
    dense1 = Dense(256,activation='relu')(tmp_simi)
    dense2 = Dense(64,activation='relu')(dense1)
    out = Dense(1,activation='linear')(dense2)
    model = Model(inputs = inputs, outputs = out)
    return model
    

In [105]:
def create_model(input_dim = 300,lstm_op_dim=50,vocab_size=4000,embed_dim=300,max_len=1000,coherence_width=50,k=10, start=3):
#     inputs = Input(shape=(max_len,))
#     embedding = Embedding(vocab_size, 300, mask_zero=True, input_length=max_len)(inputs)
    inputs = Input(shape=(max_len,input_dim))
    lstm = LSTM(lstm_op_dim, return_sequences = False)(inputs)
    op = Dense(1,activation='linear')(lstm)
    model = Model(inputs = inputs, outputs = op)    
    return model


In [45]:
model = create_model()
model.compile(optimizer='adam',
              loss='mse',lr = 0.01)

In [7]:
model.summary()

NameError: name 'model' is not defined

In [23]:
def load_training_data(train_path,essay_set = 1):
    train_path = train_path
    training_data = pandas.read_excel(train_path, delimiter='\t')
    resolved_score = training_data[training_data['essay_set'] == essay_set]['domain1_score']
    essay_ids = training_data[training_data['essay_set'] == essay_set]['essay_id']
    essays = training_data[training_data['essay_set'] == essay_set]['essay']
    essay_list = []
    for idx, essay in essays.iteritems():
        essay_list.append(clean_tokenize(essay))
    return essay_list, resolved_score.tolist(), essay_ids.tolist()

def clean_tokenize(data):
    data = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", data)
    data = re.sub(r"\'s", " \'s", data)
    data = re.sub(r"\'ve", " \'ve", data)
    data = re.sub(r"n\'t", " n\'t", data)
    data = re.sub(r"\'re", " \'re", data)
    data = re.sub(r"\'d", " \'d", data)
    data = re.sub(r"\'ll", " \'ll", data)
    data = re.sub(r",", " , ", data)
    data = re.sub(r"!", " ! ", data)
    data = re.sub(r"\(", " ( ", data)
    data = re.sub(r"\)", " ) ", data)
    data = re.sub(r"\?", " ? ", data)
    data = re.sub(r"\s{2,}", " ", data)
    data = data.lower()
    return [x.strip() for x in re.split('(\W+)?', data) if x.strip()]


def score_range(self):
    return {"1": (2, 12),"2": (1, 6),"3": (0, 3),"4": (0, 3),"5": (0, 4),"6": (0, 4),"7": (0, 30),"8": (0, 60)}

def normalize_score(self, essay_set_id, score):
    lo, hi = self.score_range[str(essay_set_id)]
    score = float(score)
    return (score - lo) / (hi - lo)


def vectorize_data(data, word_idx, sentence_size):
    E = []
    for essay in data:
        ls = max(0, sentence_size - len(essay))
        wl = []
        for w in essay:
            if w in word_idx:
                wl.append(word_idx[w])
            else:
                wl.append(0)
        wl += [0]*ls
        E.append(wl)
    return E

In [9]:
def convertword2vec(dim=300):
    word2vec = []
    word_idx = {}
    word2vec.append([0]*dim)
    count = 1
    with open('glove.6B.300d.txt') as f:
        for line in f:
            l = line.split()
            word = l[0]
            vector = list(map(float, l[1:]))
            word_idx[word] = count
            word2vec.append(vector)
            count += 1
    return word_idx, word2vec
word_idx, word2vec = convertword2vec()

In [38]:
essay_set_id = 5
essay_list, resolved_scores, essay_id = load_training_data('training_set_rel3.xlsx',essay_set_id)
vocab_size = len(word_idx) + 1
max_sent_size = 1000
vectorized_data = vectorize_data(essay_list, word_idx, max_sent_size)


/usr/lib/python3.5/re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [39]:
def convert_to_vec(vectorized_data, word2vec):
    total_vector = []
    for essay in vectorized_data:
        essay_vector = []
        for word in essay:
            essay_vector.append((word2vec[word]))
        total_vector.append(essay_vector)
    return total_vector

In [40]:
X = convert_to_vec(vectorized_data, word2vec)

In [41]:
indices = []
for i in range(len(X)):
    if(len(X[i]) > 1000):
        indices.append(i)

for i in sorted(indices,reverse=True):
    del X[i]
    del resolved_scores[i]
X = np.array(X)
        

In [42]:
print (np.shape(X))

(1805, 1000, 300)


In [43]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1000, 300)    0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 1000, 50)     70200       input_3[0][0]                    
__________________________________________________________________________________________________
lambda_81 (Lambda)              (None, 50)           0           lstm_3[0][0]                     
__________________________________________________________________________________________________
lambda_82 (Lambda)              (None, 50)           0           lstm_3[0][0]                     
__________________________________________________________________________________________________
lambda_83 

In [46]:
model.compile(optimizer=Adam(lr=0.01,decay=1e-6),
               loss='mse')
model.fit(X,resolved_scores,batch_size = 512,epochs = 100)

Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x7fcd27b91978>>
Traceback (most recent call last):
  File "/home/siddhartha.l/.local/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 1391, in __del__
    def __del__(self):
KeyboardInterrupt: 


Epoch 1/100
1805/1805 [==============================] - 17s 9ms/step - loss: 3.1620
Epoch 2/100
1805/1805 [==============================] - 12s 7ms/step - loss: 0.9788
Epoch 3/100
1805/1805 [==============================] - 11s 6ms/step - loss: 0.7579
Epoch 4/100
1805/1805 [==============================] - 11s 6ms/step - loss: 0.5860
Epoch 5/100
1805/1805 [==============================] - 10s 6ms/step - loss: 0.4342
Epoch 6/100
1805/1805 [==============================] - 11s 6ms/step - loss: 0.3461
Epoch 7/100
1805/1805 [==============================] - 11s 6ms/step - loss: 0.3355
Epoch 8/100
1805/1805 [==============================] - 12s 6ms/step - loss: 0.3345
Epoch 9/100
1805/1805 [==============================] - 11s 6ms/step - loss: 0.3139
Epoch 10/100
1805/1805 [==============================] - 11s 6ms/step - loss: 0.3086
Epoch 11/100
1805/1805 [==============================] - 11s 6ms/step - loss: 0.3059
Epoch 12/100
1805/1805 [==============================] - 11s 6

KeyboardInterrupt: 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, resolved_scores, test_size=0.3, random_state=42)
model.compile(optimizer=Adam(lr=0.01,decay=1e-6),
               loss='mse')
model.fit(X_train,y_train,batch_size = 512,epochs = 100,validation_data=(X_test,y_test))

In [46]:
predictions = model.predict_on_batch(X_test)

In [47]:
y_val=[int(np.round(a)) for a in y_test]
y_pred=[int(np.round(a)) for a in predictions]


In [45]:
model.compile(optimizer=Adam(lr=0.0001,decay=1e-6),
               loss='mse')
model.fit(X_train,y_train,batch_size = 512,epochs = 100,validation_data=(X_test,y_test))

Train on 1248 samples, validate on 535 samples
Epoch 1/100
1248/1248 [==============================] - 22s 18ms/step - loss: 0.0465 - val_loss: 0.2774
Epoch 2/100
1248/1248 [==============================] - 11s 9ms/step - loss: 0.0457 - val_loss: 0.2808
Epoch 3/100
1248/1248 [==============================] - 11s 9ms/step - loss: 0.0449 - val_loss: 0.2799
Epoch 4/100
1248/1248 [==============================] - 11s 9ms/step - loss: 0.0447 - val_loss: 0.2782
Epoch 5/100
1248/1248 [==============================] - 11s 9ms/step - loss: 0.0449 - val_loss: 0.2790
Epoch 6/100
1248/1248 [==============================] - 11s 9ms/step - loss: 0.0441 - val_loss: 0.2791
Epoch 7/100
1248/1248 [==============================] - 11s 9ms/step - loss: 0.0437 - val_loss: 0.2790
Epoch 8/100
1248/1248 [==============================] - 11s 9ms/step - loss: 0.0441 - val_loss: 0.2789
Epoch 9/100
1248/1248 [==============================] - 11s 9ms/step - loss: 0.0431 - val_loss: 0.2794
Epoch 10/100
124

KeyboardInterrupt: 

In [40]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000, 300)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 1000, 50)     70200       input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 50)           0           lstm_1[0][0]                     
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 50)           0           lstm_1[0][0]                     
__________________________________________________________________________________________________
lambda_3 (

In [48]:
from sklearn.metrics import cohen_kappa_score
print(cohen_kappa_score(y_val,y_pred))

0.5987885780213442
